In [5]:
import lightgbm
lightgbm.__version__

'3.3.2'

In [4]:
import xgboost
xgboost.__version__

'1.5.0'

In [3]:
import hyperopt
hyperopt.__version__

'0.2.7'

# HyperOpt 이용한 하이퍼 파라미터 튜닝 
1. 검색 공간 설정
2. 대체 모델을 위한 목적함수 지정
3. 최적의 파라미터를 유추

## 1. 검색공간 설정 

In [1]:
from hyperopt import hp

# 균등분포 - 최저값, 최고값, 간격 
search_space = { 'x': hp.quniform('x', -10, 10, 1), 
                'y': hp.quniform('y', -15, 15, 1) 
                }

## 2. 목적함수 지정

In [7]:
from hyperopt import STATUS_OK

def objective_func(search_space):
    x = search_space['x']
    y = search_space['y']
    retval = x**2 - 20*y

    return retval

## 3. 최적 입력 값 유추
 - fmin - 예측 값으로 최적을 찾아줌 (모두 확인하지 않음)

In [9]:
from hyperopt import fmin, tpe, Trials
import numpy as np

trial_val = Trials()

best_01 = fmin(fn=objective_func, space=search_space, algo=tpe.suggest, max_evals=5,
               trials=trial_val, rstate=np.random.default_rng(seed=0)
               )

print('best :', best_01)

100%|██████████| 5/5 [00:00<00:00, 365.73trial/s, best loss: -224.0]
best : {'x': -4.0, 'y': 12.0}


In [10]:
# max_evals 20회
best_02 = fmin(fn=objective_func, space=search_space, algo=tpe.suggest, max_evals=20,
               trials=trial_val, rstate=np.random.default_rng(seed=0)
               )

print('best :', best_02)

100%|██████████| 20/20 [00:00<00:00, 510.09trial/s, best loss: -296.0]
best : {'x': 2.0, 'y': 15.0}


In [ ]:
# 예측모델 탐색 

In [12]:
trial_val.best_trial

{'state': 2,
 'tid': 10,
 'spec': None,
 'result': {'loss': -296.0, 'status': 'ok'},
 'misc': {'tid': 10,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'x': [10], 'y': [10]},
  'vals': {'x': [2.0], 'y': [15.0]}},
 'exp_key': None,
 'owner': None,
 'version': 0,
 'book_time': datetime.datetime(2025, 1, 2, 2, 41, 18, 305000),
 'refresh_time': datetime.datetime(2025, 1, 2, 2, 41, 18, 305000)}

In [13]:
trial_val.results

[{'loss': -64.0, 'status': 'ok'},
 {'loss': -184.0, 'status': 'ok'},
 {'loss': 56.0, 'status': 'ok'},
 {'loss': -224.0, 'status': 'ok'},
 {'loss': 61.0, 'status': 'ok'},
 {'loss': -64.0, 'status': 'ok'},
 {'loss': -184.0, 'status': 'ok'},
 {'loss': 56.0, 'status': 'ok'},
 {'loss': -224.0, 'status': 'ok'},
 {'loss': 61.0, 'status': 'ok'},
 {'loss': -296.0, 'status': 'ok'},
 {'loss': -40.0, 'status': 'ok'},
 {'loss': 281.0, 'status': 'ok'},
 {'loss': 64.0, 'status': 'ok'},
 {'loss': 100.0, 'status': 'ok'},
 {'loss': 60.0, 'status': 'ok'},
 {'loss': -39.0, 'status': 'ok'},
 {'loss': 1.0, 'status': 'ok'},
 {'loss': -164.0, 'status': 'ok'},
 {'loss': 21.0, 'status': 'ok'}]

In [19]:
import pandas as pd

# result에서 loss 키값에 해당하는 밸류들을 추출하여 list로 생성. 
losses = [loss_dict['loss'] for loss_dict in trial_val.results]

# DataFrame으로 생성.
result_df = pd.DataFrame({'x': trial_val.vals['x'], 'y': trial_val.vals['y'], 'losses': losses})
result_df

,x,y,losses
0,-6.0,5.0,-64.0
1,-4.0,10.0,-184.0
2,4.0,-2.0,56.0
3,-4.0,12.0,-224.0
4,9.0,1.0,61.0
5,-6.0,5.0,-64.0
6,-4.0,10.0,-184.0
7,4.0,-2.0,56.0
8,-4.0,12.0,-224.0
9,9.0,1.0,61.0


# XGBoost 하이퍼 파라미터 최적화 

In [20]:
from hyperopt import fmin, tpe, Trials
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np 

dataset = load_breast_cancer()
cancer_df = pd.DataFrame(data = dataset.data, columns= dataset.feature_names)
cancer_df.head()


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [21]:
X_features = dataset.data
y_label = dataset.target

# 1단계 
X_train, X_test, y_train, y_test = train_test_split(X_features, y_label, test_size=0.2 ,random_state=156)

# 2단계
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=156)


In [39]:
from hyperopt import fmin, tpe, Trials, STATUS_OK
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

# 목적 함수 
def objective_func_xgb(params) : 

    model = XGBClassifier(
        n_estimator = params['n_estimators'],
        max_depth = params['max_depth'],
        learning_rate = params['learning_rate'],
        subsample = params['subsample'],
        colsample_bytree = params['colsample_bytree'],
        random_state= 42,
        eval_metric = 'logloss'
    )

    score_mean = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy').mean()

    return {'loss': -1*score_mean, 'status': STATUS_OK}

In [40]:
from hyperopt.pyll.base import scope

# 하이퍼 파라미터 검색공간 
# 정수 - quniform / 소수 - uniform

search_space = {'n_estimators': scope.int(hp.quniform('n_estimators', 50, 300, 10)), 
                'max_depth': scope.int(hp.quniform('max_depth', 3, 10, 1)),
                'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.3)),
                'subsample' : hp.uniform('subsample', 0.5, 1.0),
                'colsample_bytree' : hp.uniform('colsample_bytree', 0.5, 1.0)
                }

In [41]:
# 파라미터 유추

trials = Trials()

best_params = fmin(fn = objective_func_xgb,
                   space = search_space,
                   algo = tpe.suggest,
                   max_evals = 50,
                   trials = trials
                   )

[14:13:51] WARNING: D:\bld\xgboost-split_1637426510059\work\src\learner.cc:576: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[14:13:51] WARNING: D:\bld\xgboost-split_1637426510059\work\src\learner.cc:576: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

c:\Users\LG\.conda\envs\xgboost_env\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)

c:\Users\LG\.conda\envs\xgboost_env\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)

c:\Users\LG\.conda\envs\xgboost_env\lib\site-packages\xgboost\sklear

[14:13:51] WARNING: D:\bld\xgboost-split_1637426510059\work\src\learner.cc:576: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[14:13:51] WARNING: D:\bld\xgboost-split_1637426510059\work\src\learner.cc:576: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[14:13:51] WARNING: D:\bld\xgboost-split_1637426510059\work\src\learner.cc:576: 
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by lan

c:\Users\LG\.conda\envs\xgboost_env\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)

c:\Users\LG\.conda\envs\xgboost_env\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)

c:\Users\LG\.conda\envs\xgboost_env\lib\site-packages\xgboost\sklear

[14:13:52] WARNING: D:\bld\xgboost-split_1637426510059\work\src\learner.cc:576:  
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[14:13:52] WARNING: D:\bld\xgboost-split_1637426510059\work\src\learner.cc:576:  
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


  2%|▏         | 1/50 [00:00<00:22,  2.20trial/s, best loss: -0.9626373626373625]

...

c:\Users\LG\.conda\envs\xgboost_env\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)

c:\Users\LG\.conda\envs\xgboost_env\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)

c:\Users\LG\.conda\envs\xgboost_env\lib\site-packages\xgboost\sklear

[14:14:12] WARNING: D:\bld\xgboost-split_1637426510059\work\src\learner.cc:576:   
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[14:14:12] WARNING: D:\bld\xgboost-split_1637426510059\work\src\learner.cc:576:   
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[14:14:12] WARNING: D:\bld\xgboost-split_1637426510059\work\src\learner.cc:576:   
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used 

c:\Users\LG\.conda\envs\xgboost_env\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)

c:\Users\LG\.conda\envs\xgboost_env\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)

c:\Users\LG\.conda\envs\xgboost_env\lib\site-packages\xgboost\sklear

[14:14:12] WARNING: D:\bld\xgboost-split_1637426510059\work\src\learner.cc:576:   
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[14:14:12] WARNING: D:\bld\xgboost-split_1637426510059\work\src\learner.cc:576:   
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[14:14:12] WARNING: D:\bld\xgboost-split_1637426510059\work\src\learner.cc:576:   
Parameters: { "n_estimator" } might not be used.

  This could be a false alarm, with some parameters getting used 

c:\Users\LG\.conda\envs\xgboost_env\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)

c:\Users\LG\.conda\envs\xgboost_env\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



In [42]:
# best parameter
best_params

{'colsample_bytree': 0.6698117348241893,
 'learning_rate': 0.23968804895558135,
 'max_depth': 6.0,
 'n_estimators': 60.0,
 'subsample': 0.7431332246910244}

In [43]:
# 최적의 파라미터 적용한 모델 생성
best_model = XGBClassifier(n_estimators= int(best_params['n_estimators']), 
                           max_depth = int(best_params['max_depth']),
                           learning_rate=best_params['learning_rate'], 
                           subsample = best_params['subsample'],
                           colsample_bytree = best_params['colsample_bytree'],
                           random_state = 42,
                           eval_metric = 'logloss'
                           )

In [44]:
# 학습 
best_model.fit(X_train, y_train)

c:\Users\LG\.conda\envs\xgboost_env\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6698117348241893,
              enable_categorical=False, eval_metric='logloss', gamma=0,
              gpu_id=-1, importance_type=None, interaction_constraints='',
              learning_rate=0.23968804895558135, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=60, n_jobs=16, num_parallel_tree=1, predictor='auto',
              random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=0.7431332246910244, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [45]:
from sklearn.metrics import accuracy_score

# 예측
pred = best_model.predict(X_test)
accuracy_score(y_test, pred)

0.9473684210526315